# Linear Regression in Spark
## 1.) Basic Linear Regression in Spark
* Linear regression is a simplistic model for interpreting the relationship between features and labels so that you can then make predictions off new inputs
* Spark has a library called MLlib which allows you to perform ML tasks in Spark
* The Apache Spark site has solid documentation for all of its libraries, including MLlib and its components (e.g. linear regression, clustering, feature extraction etc.)
* Note that the Spark ML code is slightly different from standard Python (e.g. SKLearn), mostly due to the fact that the data needs to be more robustly stored and processed to allow it to run efficiently on distributed big data systems
* Below we will cover some of the core elements of linear regression in Spark
* Note that the input data here is pre-formatted for us into the Spark format (1 col for labels, 1 col for features) but in practice we will need to transform our raw data to fit this format (we will come onto this later)

In [16]:
# load libs
import findspark

# store location of spark files
findspark.init('/home/matt/spark-3.0.2-bin-hadoop3.2')

# load libs
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

# start new session
spark = SparkSession.builder.appName('lreg').getOrCreate()

# load train data
data = spark.read.format('libsvm').load('Data/sample_linear_regression_data.txt')

# peek at data
data.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



### Train/Test Split
* We can use **randomSplit()** to separate our train and test data randomly
* It's important that this is random otherwise we may capture in build ordering/sorting from our raw data
* All Spark dataframe objects have this method available to them

In [17]:
# randomly split data into train/test
train, test = data.randomSplit([0.7, 0.3])

# create model instance
# you can call these what you like
# but they must match your input df
# there are many more parameters you can use
# shift + tab to see the options available
lr = LinearRegression(featuresCol='features',
                      labelCol='label',
                      predictionCol='prediction')

# create model var
lr_model = lr.fit(train)

# check co-efficients
lr_model.coefficients

DenseVector([-0.0591, 0.9713, -0.949, 2.5666, 1.4099, 1.0595, 0.2065, -0.3038, -1.0501, 1.6254])

### Evaluating our Model
* Once a model is built in Spark, you can access all of the evaluation metrics you could need
* R2, MSE, RMSE, MAE etc. are all available of the model object
* You can of course calculate the same metrics for your test data once you've applied your trained model to the test features to make predictions

In [18]:
# check y-intercept
lr_model.intercept

0.641994426807195

In [19]:
# get summary of model
train_summary = lr_model.summary

# show r2 value
# use shift + tab to show all options
train_summary.r2

0.03870866789062066

In [20]:
# show MSE
train_summary.meanSquaredError

107.75977379756242

In [21]:
# predict labels based on test features
# test_results is essentially y_pred
test_results = lr_model.evaluate(test)

# assess scores of test predictions
test_results.rootMeanSquaredError

9.791620046000943

### Making Predictions
* The above code showed us how well our labelled test data fitted to our linear model
* However, we can also make new predictions using the model and our test features
* Then we can assess the prediction score of our model by comparing predicted values to actual values

In [24]:
# extract test data without labels
unlabelled_data = test.select('features')

# show data
unlabelled_data.show(3)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 3 rows



In [25]:
# make predictions using model and unlabelled test data
predictions = lr_model.transform(unlabelled_data)

# show predictions
predictions.show(3)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|(10,[0,1,2,3,4,5,...|-1.347831113305837|
|(10,[0,1,2,3,4,5,...|0.2744128758073392|
|(10,[0,1,2,3,4,5,...|3.0104295755815533|
+--------------------+------------------+
only showing top 3 rows



## 2.) Evaluation Metrics
### Mean Absolute Error (MAE)
* Simple absolute value of error
* For each instance, calculate error between actual and predicted value
* Convert this to absolute value to remove -ve and +ve cancelling out
* Average the absolute error across all instances
* Tells you how far away from your actual results your predictions are using your model

### Mean Squared Error (MSE)
* Slight improvement on the above method
* Same process as MAE except that you square the difference between actual and predicted
* This penalizes larger errors in your model so that when your predictions are way off, this is highlighted further
* The only issue here is that because you've squared your values, you are no longer looking at the raw scales for your values, making the MSE harder to interpret literally

### Root Mean Squared Error (RMSE)
* As such, the RMSE improves upon the above method
* It does so by finding the square root of the MSE (i.e. finds error, squares it, averages this across all instances and then square roots the final value)
* This means that it successfully penalizes larger error predictions whilst also reverting the final RMSE value to a scale which can be interpreted directly in relation to your raw data scales
* As such, this model is the most popular of the 3 mentioned

### R-Squared (r2)
* This is not technically an error metric like the above
* a.k.a. **co-efficient of regression**
* It's essentially a measure of how much variance your model accounts for
* It ranges between 0 and 1 (where a value of 0.9 means your model describes 90% of the variance of the data)
* In general the higher the better
* There are also variants, such as adjusted r2 value
    * r2 simply tells you how much of the variance in your outputs is described by your inputs
    * The higher the r2, the better your model is at interpreting the relationship between inputs and outputs
    * However, if you add more variables to your model, the r2 squared value will increase regardless of whether or not the new variables are significantly impacting variance in your outputs
    * As such, adjusted r2 will penalize your model (i.e. return a lower value) if new variables are added which do not improve your model or significantly affect your outputs
    * Therefore, for multiple regression models, adjust r2 should always be used to avoid the raw r2 value being 'gamed' by simply adding more variables
    
## 3.) Real World Linear Regression
### Load in Data
* Most real world data won't be in a neat, Spark compatible format already
* i.e. it won't have 1 column of labels and 1 column of all features
* As such, we will explore how to transform data into a Spark-friendly format below

In [26]:
# read in data
df = spark.read.csv('Data/Ecommerce_Customers.csv', inferSchema=True, header=True)

# show schema
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [29]:
# peek at data
for item in df.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


### Data as Vectors
* As we saw above, Spark ML requires data to have features and labels in 1 column each
* Below we use the VectorAssembler to convert our range of features into a single features column
* We can then perform standard processes on this data (i.e. train/test split, model fitting etc.)

In [31]:
# load libraries to process data into vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

# create an assembler which converts features into single feature column/vector
# this tells the assembler what to expect, it doesn't actually create the output
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App',
                                       'Time on Website', 'Length of Membership'],
                            outputCol='features')

# create single output vector from multiple input features
output = assembler.transform(df)

# show output
# see 'features=[a, b, c, d]' where a single vector
# contains one value per original input feature
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [33]:
# select final data after above process
# simply features and labels in 2 columns
final = output.select('features', 'Yearly Amount Spent')

# show final data
final.show(3)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
+--------------------+-------------------+
only showing top 3 rows



In [36]:
# split train and test data
train, test = final.randomSplit([0.7, 0.3])

# create linear regression model
lr = LinearRegression(featuresCol='features',
                      labelCol='Yearly Amount Spent',
                      predictionCol='predictions')

# fit model to training data
lr_model = lr.fit(train)

# evaluate test results
# i.e. how good is our trained model on our test data
test_results = lr_model.evaluate(test)

# check residuals
# i.e. actual - test variance
test_results.residuals.show(3)

+-------------------+
|          residuals|
+-------------------+
|0.07758416715586236|
| 10.170815702707046|
| 6.1357435255481505|
+-------------------+
only showing top 3 rows



### Model Evaluation
* This model is a pretty good one, our r2 value is 98.6% which is very strong
* Our RMSE is also quite low compared to the actual units of our data (i.e. RMSE = 9.5 compared to our data mean of 499.3)
* It's always worth comparing scoring metrics to your actual values where possible (although obviously if you're using e.g. MSE then the scale of the metric isn't directly comparable)
* Note that the score here is very high and often for simple models (i.e. linear regression) it's unlikely your model will ever be this good. If you encounter this in the real world, always double check your results, it's only because this dataset is a model that the algorithm fits so well

In [37]:
# show RMSE
test_results.rootMeanSquaredError

9.460425615133678

In [38]:
# show r2
test_results.r2

0.9855444227766031

In [39]:
# compare to original data
final.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [40]:
# extract features only from test to create unlabelled data
unlabelled_data = test.select('features')

# make predictions of labels using test features
predictions = lr_model.transform(unlabelled_data)

# check outputs
predictions.show(3)

+--------------------+------------------+
|            features|       predictions|
+--------------------+------------------+
|[30.5743636841713...| 441.9868295909098|
|[30.7377203726281...|451.60992649352283|
|[30.9716756438877...|488.50286623134457|
+--------------------+------------------+
only showing top 3 rows

